# Compare Peter Zeidler's NGC 346 cube with ESO data archive cube of same observations

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from  astropy.table import Table
import seaborn as sns
from mpdaf.obj import Cube
import regions
sns.set_context("talk")

## Spectrum of the whole cube

In [ ]:
datapath = Path.home() / "Work/Muse-Hii-Data/SMC-NGC-346"
cubeP = Cube(str(datapath / "PeterZeidler" / "DATACUBE_FINAL_fwhm_cor.fits"))

Also load the standard ESO cube for comparison

In [ ]:
cubeE = Cube(str(datapath / "ADP.2017-10-16T11_04_19.247.fits"))

Extract full-cube spectra. This next cell takes several seconds to execute.

In [ ]:
specP = cubeP.sum(axis=(1, 2))
specE = cubeE.sum(axis=(1, 2))

### Global comparison of full spectrum

First look at the full wav range on a log intensity scale

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
specP.plot(ax=ax, label="Peter")
specE.plot(ax=ax, label="ESO")
ax.legend()
ax.set(
    yscale="log",
)
...;

So Peter cube is consistently brighter, presumably due to better photometry. Also has more sky features visible in the spectrum.

Next, look ast the ratio between the two on a linear scale:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
(specP / specE).plot(ax=ax, label="Ratio")
ax.legend()
avratio = np.nanmedian(specP.data / specE.data)
ax.axhline(avratio, color='r', zorder=-100)
ax.set(
    ylim=[0.0, None],
)
...;

In [ ]:
avratio

### Split into multiple wavelength sections

Choose 12 sections, which means that each section spans about 400 Angstrom.

In [ ]:
wave_min, wave_max = specP.wave.get_range()
n_sections = 12
d_wave = (wave_max - wave_min) / n_sections
start_waves = wave_min + np.arange(n_sections) * d_wave
start_waves

Scale the ESO cube by the median ratio, so we can compare in more detail.

In [ ]:
spec_sections_P = [specP.subspec(w0, w0 + d_wave) for w0 in start_waves]
spec_sections_E = [avratio * specE.subspec(w0, w0 + d_wave) for w0 in start_waves]

First compare on a logarithmic flux scale.

In [ ]:
fig, axes = plt.subplots(n_ranges, 1, figsize=(12,  2 * n_ranges))
for section_P, section_E, ax in zip(spec_sections_P, spec_sections_E,  axes):
    section_P.plot(ax=ax)
    section_E.plot(ax=ax)
    ax.set(yscale="log", ylabel="", xlabel="")
fig.tight_layout()

So, the strongest discrepancy is the [O I] 5577 line, which will be extremely weak  from  the nebula, but is strong from the night sky. This is almost completely removed in the ESO cube but is still strong in the Peter cube.

The Peter cube is also higher at the position of real nebula lines, such as H beta and [O III].  In these cases, I think it is not true night sky emission, but is rather the outer parts of the nebula that are being picked up. These are oversubtracted in the ESO cube, resulting in the lines being apparently seen in absorption in fainter spaxels of the cube,  The Peter cube does not have this problem.

Next, we look at the ratio of the two cubes, zooming in on the range +/-20% so we can see the weak night sky lines and atmospheric absorption.

In [ ]:
fig, axes = plt.subplots(n_ranges, 1, figsize=(12,  2 * n_ranges))
for section_P, section_E, ax in zip(spec_sections_P, spec_sections_E,  axes):
    (section_P / section_E ).plot(ax=ax)
    ax.set(ylim=[0.8, 1.2], ylabel="", xlabel="")
axes[0].set_title("Ratio: Peter / ESO")
fig.tight_layout()

We see a lot of weak absorption features with EW of less than an Angstrom. Also lots of emission lines. In most sections, it is possible to distinguish the two, but at longer wavelengths this is more difficult.

We could look at the model sky spectra to try and help with this.  We probably need to be able to separate the sky emission from the absorption, since the first is additive, while the second is multiplicative. **Actually, this might be a way of doing it:** in principal the absorption lines should be spatially constant in a map of `cube/avspec`, whereas the emission lines should be constant in a map of `(cube - avspec)`

### Fitting the continuum

I had an idea that we can use sigma clipping in the fit so that we do not have to bother with masking out the emission lines first.  I will try this on the entire cube first, since I  am not sure how fast it is going to be to apply it pixel by pixel.

- [ ] TODO do the fits

## Look at particular regions
To start with I will use the bowshock ones that I made for Jesus to use for the Spitzer spectra. Later, I should do some that concentrate more on the filaments.

In [ ]:
small_data_path = Path.cwd().parent.parent / "data"
region_file = "ngc346-jesus-muse-pixels.reg"
pixel_regions = regions.Regions.read(small_data_path / region_file)

In [ ]:
region_dict = {rg.meta["label"]: rg for rg in pixel_regions}
Table(rows=region_dict.items())

In [ ]:
nlam, ny, nx = cubeP.shape

In [ ]:
fig, ax = plt.subplots()
patches = []
for label, reg in region_dict.items():
    patches.append(
        reg.plot(ax=ax, facecolor='none', edgecolor='red', lw=2, label=label)
    )
ax.legend(handles=patches)
ax.set(
    xlim=[0, nx],
    ylim=[0, ny],
)
ax.set_aspect('equal')

In [ ]:
cubeE.shape[1:]

In [ ]:
region_dict["BS"].to_mask().to_image((ny, nx)).reshape((1, ny, nx)) * cubeP.data.data

### Divide  by the mean cube